# Fetch json_widget branch from github
https://github.com/mocquin/physipy/tree/json_widget

# 5 line tutorial on Quantity from [physipy](https://github.com/mocquin/physipy)

In [1]:
from physipy import quantify, Quantity, Dimension
from physipy import m, s

# A quantity is the association of a value and a Dimension object (a dictionnary)
five_meters = 5*m
# m itself is a quantity with value 1 and dimension "L" for lenght
print(f'{m} : has value {m.value} and dimension {m.dimension}, or as a dict : {m.dimension.dim_dict}')
print()
print(f'{five_meters**2} (5*m squared) : has value {(five_meters**2).value}, and dimension {(five_meters**2).dimension}, or as a dict : {(five_meters**2).dimension.dim_dict}')

1 m : has value 1 and dimension L, or as a dict : {'L': 1, 'M': 0, 'T': 0, 'I': 0, 'theta': 0, 'N': 0, 'J': 0, 'RAD': 0, 'SR': 0}

25 m**2 (5*m squared) : has value 25, and dimension L**2, or as a dict : {'L': 2, 'M': 0, 'T': 0, 'I': 0, 'theta': 0, 'N': 0, 'J': 0, 'RAD': 0, 'SR': 0}


In [2]:
from physipy.quantity.utils import quantity_json_encoder, quantity_json_decoder

def quantity_to_json(q, widget):
    return quantity_json_encoder(q)

def quantity_from_json(q, widget):
    return quantity_json_decoder(q)

# example 
quantity_json_encoder(5*s) # five seconds

in quantity_json_encoder with 5 s
in _to_json_dict


{'Quantity': True,
 'value': 5,
 'dim_dict': {'L': 0,
  'M': 0,
  'T': 1,
  'I': 0,
  'theta': 0,
  'N': 0,
  'J': 0,
  'RAD': 0,
  'SR': 0}}

# attempt to create a QuantityFloatSlider

In [12]:
from traitlets import (
    Instance, Unicode, CFloat, Bool, CaselessStrEnum, Tuple, TraitError, validate, default, 
)
from traitlets.traitlets import _validate_bounds

from ipywidgets.widgets.widget_description import DescriptionWidget
from ipywidgets.widgets.trait_types import InstanceDict, NumberFormat
from ipywidgets.widgets.valuewidget import ValueWidget
from ipywidgets.widgets.widget import register, widget_serialization
from ipywidgets.widgets.widget_core import CoreWidget
from ipywidgets.widgets.widget_int import ProgressStyle


from traitlets import (
    Instance, Unicode, CFloat, Bool, CaselessStrEnum, Tuple, TraitError, validate, default, 
)
from traitlets.traitlets import _validate_bounds

from ipywidgets.widgets.widget_description import DescriptionWidget
from ipywidgets.widgets.trait_types import InstanceDict, NumberFormat
from ipywidgets.widgets.valuewidget import ValueWidget
from ipywidgets.widgets.widget import register, widget_serialization
from ipywidgets.widgets.widget_core import CoreWidget
from ipywidgets.widgets.widget_int import ProgressStyle



from ast import literal_eval
import contextlib
import inspect
import os
import re
import sys
import types
import enum
from warnings import warn, warn_explicit

from traitlets.utils.getargspec import getargspec
from traitlets.utils.importstring import import_item
from traitlets.utils.sentinel import Sentinel
from traitlets.utils.bunch import Bunch
from traitlets.utils.descriptions import describe, class_of, add_article, repr_type
from traitlets import TraitType

SequenceTypes = (list, tuple, set, frozenset)

# backward compatibility, use to differ between Python 2 and 3.
ClassTypes = (type,)

# exports:

__all__ = [
    "default",
    "validate",
    "observe",
    "observe_compat",
    "link",
    "directional_link",
    "dlink",
    "Undefined",
    "All",
    "NoDefaultSpecified",
    "TraitError",
    "HasDescriptors",
    "HasTraits",
    "MetaHasDescriptors",
    "MetaHasTraits",
    "BaseDescriptor",
    "TraitType",
    "parse_notifier_name",
]

# any TraitType subclass (that doesn't start with _) will be added automatically

#-----------------------------------------------------------------------------
# Basic classes
#-----------------------------------------------------------------------------


Undefined = Sentinel('Undefined', 'traitlets',
'''
Used in Traitlets to specify that no defaults are set in kwargs
'''
)



from traitlets import TraitType



class QuantityTrait(TraitType):
    """A trait for Quantity.
    """

    #default_value = 
    info_text = 'a Quantity object'

    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        print("in init of QuantityTrait")
        default_value = quantify(default_value)
        default_dim = default_value.dimension
        self.min = kwargs.pop('min', Quantity(-float('inf'),
                                              default_dim))
        self.max = kwargs.pop('max', Quantity(float('inf'),
                                             default_dim))
        super(QuantityTrait, self).__init__(default_value=default_value,
                                    allow_none=allow_none, **kwargs)
    
    def validate(self, obj, value):
        if not isinstance(value, Quantity):
            self.error(obj, value)
        return _validate_bounds(self, obj, value)

#print(QuantityTrait(q).min)
#print(QuantityTrait(q).max)
#print(QuantityTrait(4).min)
#print(type(QuantityTrait(4).min))

In [13]:
class CQuantityTrait(QuantityTrait):
    """A casting version of the QuantityTrait trait."""

    def validate(self, obj, value):
        try:
            value = quantify(value)
        except Exception:
            self.error(obj, value)
        return _validate_bounds(self, obj, value)

#print(CQuantityTrait(q).min)
#print(CQuantityTrait(q).max)
#print(CQuantityTrait(4).min)
#print(type(CQuantityTrait(4).min))
#
#cq = CQuantityTrait(q)
#print(cq)
#print(cq.default_value)

In [14]:
class _QuantityTrait(DescriptionWidget, ValueWidget, CoreWidget):
    #value = CQuantityTrait(0.0, help="Quantity value").tag(sync=True)

    def __init__(self, value=None, **kwargs):
        if value is not None:
            kwargs['value'] = value
            self.__class__.value = CQuantityTrait(value,
                                                  help="Quantity value").tag(sync=True,
                                                                             to_json=quantity_to_json,
                                                                             from_json=quantity_from_json)
            self.__class__.step = CQuantityTrait(Quantity(0.1,
                                                          value.dimension),
                                                 allow_none=True, 
                                                 help="Minimum step to increment the value").tag(sync=True, 
                                                                                                 to_json=quantity_to_json,
                                                                                                 from_json=quantity_from_json)
    
        super().__init__(**kwargs)

In [15]:
class _BoundedQuantity(_QuantityTrait):
    

    @validate('value')
    def _validate_value(self, proposal):
        """Cap and floor value"""
        value = proposal['value']
        self.max = CQuantityTrait(Quantity(100,
                                           value.dimension, help="Max value")).tag(sync=True,
                                                                                   to_json=quantity_to_json,
                                                                                   from_json=quantity_from_json)
        self.min = CQuantityTrait(Quantity(0, 
                                           value.dimension, help="Min value")).tag(sync=True,
                                                                                   to_json=quantity_to_json,
                                                                                   from_json=quantity_from_json)
        if self.min > value or self.max < value:
            value = min(max(value, self.min), self.max)
        return value

    @validate('min')
    def _validate_min(self, proposal):
        """Enforce min <= value <= max"""
        min = quantify(proposal['value'])
        if min > self.max:
            raise TraitError('Setting min > max')
        if min > self.value:
            self.value = min
        return min

    @validate('max')
    def _validate_max(self, proposal):
        """Enforce min <= value <= max"""
        max = quantify(proposal['value'])
        if max < self.min:
            raise TraitError('setting max < min')
        if max < self.value:
            self.value = max
        return max


In [18]:
from ipywidgets.widgets.widget_int import IntText, BoundedIntText, IntSlider, IntProgress, IntRangeSlider, Play, SliderStyle

@register
class QuantitySlider(_BoundedQuantity):

    _view_name = Unicode('FloatSliderView').tag(sync=True)
    _model_name = Unicode('FloatSliderModel').tag(sync=True)
    orientation = CaselessStrEnum(values=['horizontal', 'vertical'],
        default_value='horizontal', help="Vertical or horizontal.").tag(sync=True)
    readout = Bool(True, help="Display the current value of the slider next to it.").tag(sync=True)
    #readout_format = NumberFormat(
    #    '.2f', help="Format for the readout").tag(sync=True)
    continuous_update = Bool(True, help="Update the value of the widget as the user is holding the slider.").tag(sync=True)
    disabled = Bool(False, help="Enable or disable user changes").tag(sync=True)

    #style = InstanceDict(SliderStyle).tag(sync=True, **widget_serialization)


In [23]:
# import a meter "m"
from physipy.quickstart import m, s
qs = QuantitySlider(0.5*m, min=0*m, max=10*m)
qs

in init of QuantityTrait
in init of QuantityTrait
in quantity_json_encoder with 0.5 m
in _to_json_dict
in quantity_json_encoder with 0.5 m
in _to_json_dict


QuantitySlider(value=<Quantity : 0.5 m>, step=<Quantity : 0.5 m>)

In [30]:

print(qs.keys)
print(qs.step)
print(qs.value)
print(qs.readout)
print(qs.value)
qs.value = 10*m
print(qs.value)
# this should raise a DimensionError, and it does ! qs.value = 10*s


['_dom_classes', '_model_module', '_model_module_version', '_model_name', '_view_count', '_view_module', '_view_module_version', '_view_name', 'continuous_update', 'description', 'description_tooltip', 'disabled', 'layout', 'orientation', 'readout', 'step', 'style', 'value']
10 m
10 m
True
10 m
10 m


In [25]:
qs

QuantitySlider(value=<Quantity : 10 m>, step=<Quantity : 10 m>)

In [26]:
import ipywidgets

In [28]:
ipywidgets.interactive(2)

interactive(children=(Output(),), _dom_classes=('widget-interact',))